In [ ]:
pip install cassandra-driver

In [ ]:
from cassandra.cluster import Cluster

# Connect to Cassandra Cluster
def connect_to_cassandra():
    cluster = Cluster(["127.0.0.1"])  # Change IP if running on a remote node
    session = cluster.connect()  # Connect without setting a keyspace

    # ✅ Create keyspace before setting it
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS mykeyspace
        WITH replication = {'class': 'SimpleStrategy', 'replication_factor': '1'}
    """)
    print("Keyspace created successfully!")

    session.set_keyspace("mykeyspace")  # Now it's safe to set the keyspace
    return session  # Return the session object

session = connect_to_cassandra()

In [ ]:
def create_table(session):
    session.execute("""
        CREATE TABLE IF NOT EXISTS mykeyspace.users (
            id UUID PRIMARY KEY,
            name TEXT,
            age INT,
            city TEXT
        )
    """)
    print("Table created successfully!")

create_table(session)

In [ ]:
import uuid

# Insert Data 
def insert_data(session, user_id, name, age, city):
   session.execute("""
        INSERT INTO users (id, name, age, city)
        VALUES (%s, %s, %s, %s)
    """, (uuid.UUID(user_id), name, age, city))

insert_data(session, str(uuid.uuid4()), "John", 23, "New York")  # ✅ Pass a valid UUID
print("Data Inserted")


In [ ]:
# Read Data
def read_data(session):
    rows = session.execute("SELECT * FROM mykeyspace.users")
    for row in rows:
        print(row)

read_data(session)

In [ ]:
def delete_row(session, user_id):
    session.execute("DELETE FROM users WHERE id = %s", (uuid.UUID(user_id),))
    print(f"Row with ID {user_id} deleted successfully!")

delete_row(session,'06b1647f-b009-4e81-8a9e-a05d9332f7ac')

In [ ]:
def delete_table(session,table_name):
    session.execute(f"DROP TABLE IF EXISTS {table_name}")
    print(f"Table {table_name} deleted successfully!")

delete_table(session,"users")

In [ ]:
# ✅ Delete Entire Keyspace
def delete_keyspace(session,keyspace):
    session.execute(f"DROP KEYSPACE IF EXISTS {keyspace}")
    print(f"Keyspace {keyspace} deleted successfully!")

delete_keyspace(session,"mykeyspace")